# PROJET IA for HumanForYou - Entraînement et Évaluation du Modèle

|Auteur|
|---|
|G. DUBOYS DE LAVIGERIE|
|T. VILLETTE|
|O. BOUSSARD|
|A. BRICON|

## Objectifs du Notebook

Ce notebook a pour objectif de développer plusieurs modèles d'IA en utilisant les données prétraitées et de les évaluer sur un ensemble de test. Les étapes comprennent :

1. **Division des données :** Séparation des données en ensembles d'entraînement et de test.
2. **Entraînement des modèles :** Développement de plusieurs modèles d'IA avec les données d'entraînement.
3. **Évaluation des modèles :** Mesure des performances de chaque modèle sur l'ensemble de test.
4. **Sélection du meilleur modèle :** Analyse des résultats pour choisir le modèle offrant les performances les plus prometteuses.

## Attendus

À la fin de ce notebook, le modèle sera prêt à être déployé, accompagné d'une évaluation détaillée de ses performances.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir exécuté le notebook `data_preprocessing.ipynb` pour garantir que les données sont prêtes pour l'entraînement du modèle.

### Préparation de l'environnement

In [ ]:
# Compatibilité entre Python 2 et Python 3
from __future__ import division, print_function, unicode_literals

# Imports nécessaires
import numpy as np
import os

# Assurer la stabilité du notebook entre différentes exécutions
np.random.seed(42)

# Affichage de figures directement dans le notebook
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

# Ajustements des paramètres de l'affichage des figures
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Définition du chemin où sauver les figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "workflowDS"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)  # Assurez-vous que le dossier existe

# Fonction pour sauvegarder les figures
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Sauvegarde de la figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignorer les avertissements inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### Importation des données traitées réalisée dans le notebook `data_preprocessing.ipynb`

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) # Retirer la limite de colonnes à afficher

def load_preprocessed_data(file_path='../datasets/data_preprocessing.csv'):
    return pd.read_csv(file_path)

# Charger les données prétraitées
result = load_preprocessed_data()
result.head()

### Division des données (jeu d'entraînement, jeu de test)
Cette étape consiste à séparer les données en un jeu d'entraînement (80%) et un jeu de test (20%) afin d'évaluer la capacité du modèle à généraliser sur de nouvelles données. 

- Le **jeu d'entraînement** est utilisé pour former le modèle, lui fournissant une quantité suffisante de données pour apprendre les motifs et les relations présents dans nos données.

- Le **jeu de test** permet de mesurer la performance du modèle sur des données non vues pendant l'entraînement, offrant ainsi une évaluation impartiale de sa capacité de généralisation.

Le choix de la division 80/20 n'est pas arbitraire. Il s'agit d'une règle empirique bien établie dans le domaine de l'apprentissage machine, offrant un équilibre optimal entre la taille de l'ensemble d'entraînement nécessaire pour un apprentissage efficace et la capacité à évaluer le modèle de manière significative sur le jeu de test.


In [ ]:
from sklearn.model_selection import train_test_split

# Supposons que nous voulons prédire 'cat_nom__Attrition_Yes'
y = result['cat_ord__Attrition']
X = result.drop(['cat_ord__Attrition'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Affichage d'un graphique
fig, ax = plt.subplots(figsize=(2, 5))

rect_train = plt.Rectangle((0, 0), 1, 0.8, color='red', alpha=0.5, label='Entraînement')
rect_test = plt.Rectangle((0, 0.8), 1, 0.2, color='blue', alpha=0.5, label='Test')

ax.add_patch(rect_train)
ax.add_patch(rect_test)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Répartition des Données")

ax.text(0.5, 0.4, f'{len(X_train)}', ha='center', va='center', color='black', fontsize=8)
ax.text(0.5, 0.9, f'{len(X_test)}', ha='center', va='center', color='black', fontsize=8)

ax.legend(loc='center')

plt.show()


### Choisir et Entraîner un modèle

Avant de plonger dans les détails, il est essentiel de comprendre comment choisir, entraîner et évaluer un modèle. Une étape cruciale consiste à analyser les performances du modèle sur un ensemble de test et à utiliser des techniques comme la validation croisée pour garantir une évaluation robuste du modèle.

**Fonction de validation croisée**

La fonction `train_evaluate_model` ci-dessous illustre un processus générique d'entraînement et d'évaluation de modèles de régression. Cela garantit une approche standardisée dans notre notebook, simplifiant ainsi le processus et assurant une cohérence d'évaluation, indépendamment de l'algorithme sous-jacent.

En utilisant la fonction `train_evaluate_model`, nous appliquons la validation croisée avec deux métriques :
- **RMSE (Root Mean Square Error)** : Mesure la moyenne des erreurs au carré entre prédictions et valeurs réelles, avec prise de la racine carrée de cette moyenne.
- **MAE (Mean Absolute Error)** : Autre métrique mesurant la moyenne des valeurs absolues des erreurs entre prédictions et valeurs réelles.

Cette approche renforce l'évaluation des modèles en utilisant `scikit-learn` pour calculer le RMSE, la MAE, et la validation croisée, fournissant une évaluation complète de la performance du modèle.

Il convient de noter que nous utilisons cette fonction pour chacun de nos modèles, assurant ainsi une comparaison équitable des performances entre les différentes approches algorithmiques que nous explorons.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def train_model(trainer):
    # Entraînement du modèle
    trainer.fit(X_train, y_train)
    
    some_data = X_train.iloc[:5]
    some_labels = y_train.iloc[:5]

    # Et on effectue la prédiction :
    print("Predictions:", trainer.predict(some_data))
    print("Labels:", list(some_labels)) # vraies valeurs

    # Prédictions sur les données de test
    trainer_predictions = trainer.predict(X_test)
    # Évaluation des prédictions sur les données de test
    trainer_mse = mean_squared_error(y_test, trainer_predictions)
    trainer_rmse = np.sqrt(trainer_mse)

    # Affichage des métriques d'évaluation
    print("MSE:", trainer_mse)
    print("RMSE:", trainer_rmse)

    # Validation croisée pour évaluer la performance du modèle
    scores = cross_val_score(trainer, X_test, y_test,
                             scoring="neg_mean_squared_error", cv=10)
    trainer_rmse_scores = np.sqrt(-scores)

    # Affichage des scores de validation croisée
    print("Cross-Validation Scores:", trainer_rmse_scores)

    # Affichage de la performance moyenne et de la variabilité du modèle
    print("Mean RMSE:", trainer_rmse_scores.mean())
    print("RMSE Standard Deviation:", trainer_rmse_scores.std())

#### Modèle de régrission linéaire
Entraînons un modèle de [_régression linéaire_](https://fr.wikipedia.org/wiki/R%C3%A9gression_lin%C3%A9aire) avec nos données.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
train_model(lin_reg)

#### Modèle de régression par arbre de décision (DecisionTreeRegressor) 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create a DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
train_model(tree_reg)


### Modèle SVR

In [ ]:
from sklearn.svm import SVR

svr_reg = SVR(C=9.318742350231167, gamma=0.09849250205191949)
train_model(svr_reg)

### Modèle RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_depth=15, max_features='log2', n_estimators=26,
                      random_state=42)
train_model(forest_reg)

## Gridsearch
Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour un modèle de régression RandomForestRegressor.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [26], 'max_features': ['log2'], 'max_depth': [15]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
 
# 5 sous-jeux de cross-val, ça fait en tout (12+6)*5=90 tours d'entraînement 
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train,y_train )

print(grid_search.best_params_)
print(grid_search.best_estimator_)

In [ ]:
feature_importance = grid_search.best_estimator_.feature_importances_
attributes = result.columns
sorted(zip(feature_importance, attributes), reverse=True)

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle SVR.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [
    # essaye 12 (3×4) combinaisons des hyperparametres
    {'kernel': ['linear'], 'C': [10., 30., 100.]},
    # puis essaye 6 (2×3) combinaisons avec le noyau RBF
    {'kernel': ['rbf'], 'C': [10., 30., 100.],'gamma': [0.3, 1.0, 3.0]},
]

svr_reg = SVR()

# 5 sous-jeux de cross-val, ça fait en tout (8+6*6)*5=350 tours d'entraînement 
grid_search = GridSearchCV(svr_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', verbose=2, n_jobs=4)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle de régression linéaire.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

param_grid = [
    {'fit_intercept': [True, False]}
]

lin_reg = LinearRegression()

grid_search = GridSearchCV(lin_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)

Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour le modèle de régression par arbre de décision (DecisionTreeRegressor).

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

param_grid = [
    {'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 5]}
]

tree_reg = DecisionTreeRegressor(random_state=42)

grid_search = GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)


Nous avons effectué une recherche aléatoire (Randomized Search) avec la méthode RandomizedSearchCV afin de trouver les meilleurs hyperparamètres pour les  modèles SVR (Support Vector Regressor) et RandomForestRegressor. Nous n'avons testé cette méthode que sur ces 2 modèles car c'est les plus efficaces.
La recherch aléatoire est  une technique de recherche qui consiste à choisir un élément au hasard dans une population pour déterminer si cela peut être considéré comme le meilleur ou le pire exemple possible. Nous avons testé cette méthode pour vérifier que les résultats de la recherche par grille étaient correctes.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import uniform, randint

# Paramètres pour SVR
param_distributions_svr = {
    'C': uniform(0.1, 10),
    'gamma': uniform(0.01, 1),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

svr = SVR()

random_search_svr = RandomizedSearchCV(svr, param_distributions=param_distributions_svr, 
                                       n_iter=500, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search_svr.fit(X_train, y_train)

print(random_search_svr.best_params_)
print(random_search_svr.best_estimator_)

# Paramètres pour RandomForestRegressor
param_distributions_rf = {
    'n_estimators': randint(10, 200),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': randint(2, 20),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10)
}

rf = RandomForestRegressor(random_state=42)

random_search_rf = RandomizedSearchCV(rf, param_distributions=param_distributions_rf, 
                                      n_iter=500, cv=5, scoring='neg_mean_squared_error', random_state=42)
random_search_rf.fit(X_train, y_train)

print(random_search_rf.best_params_)
print(random_search_rf.best_estimator_)

Mélange aléatoire des données.

In [ ]:
import numpy as np

shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

### Validation croisée (cross-validation)
La validation croisée est une technique d'évaluation des modèles. Elle permet d'évaluer les performances des modèles d'apprentissage automatique.
On utilise la méthode K -Folds. Cette méthode consiste séparer les données de manière aléatoire en K folds K faisant référence au nombre de groupes dans lequel l’échantillon sera divisé.Le modèle est entraîné sur une partie des données et testé sur une autre.

In [ ]:
y_train_yes = (y_train > 0)
y_test_yes = (y_test > 0 )

from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=1000, random_state=42)
sgd_clf.fit(X_train, y_train_yes)

sgd_clf.predict(X_test)


# cross-val a la mano
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)

for train_index, test_index in skfolds.split(X_train, y_train_yes):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = (y_train_yes[train_index])
    X_test_fold = X_train[test_index]
    y_test_fold = (y_train_yes[test_index])

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

# ⨯-val avec scikit-learn
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, y_train_yes,cv=3 ,scoring="accuracy" )    


In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

In [ ]:
never_5_clf = Never5Classifier()
predictions = never_5_clf.predict(X_train)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_yes, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix =  confusion_matrix(y_train_yes, y_train_pred)

from sklearn.metrics import precision_score, recall_score
precision_score(y_train_yes, y_train_pred)

In [ ]:
recall_score(y_train_yes, y_train_pred)

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_train_yes, y_train_pred)

In [ ]:
y_scores = sgd_clf.decision_function(X_train)

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_yes, cv=3,
                             method="decision_function")

from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_yes, y_scores)


def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b-", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.ylim([0, 1])

plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.xlim([-700000, 700000])
plt.show()

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "k-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)
plt.show()

In [ ]:
y_train_pred_90 = (y_scores > 70000)

precision_score(y_train_yes, y_train_pred_90), recall_score(y_train_yes, y_train_pred_90)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_yes, y_scores)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

# Supposons que y_true sont vos vraies étiquettes binaires, et y_scores sont les scores de décision de votre modèle
auc_roc = roc_auc_score(y_train_yes, y_scores)

print("AUC-ROC: ", auc_roc)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

forest_clf = RandomForestClassifier(random_state=42, n_estimators=10)
y_probas_forest = cross_val_predict(forest_clf, X_train, y_train_yes, cv=3,
                                    method="predict_proba")

In [ ]:
y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_train_yes,y_scores_forest)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.legend(loc="lower right", fontsize=16)
plt.show()

In [ ]:
auc = roc_auc_score(y_train_yes, y_scores_forest)
print("AUC: ", auc)

In [ ]:
y_pred_forest = (y_scores_forest > 0.5)
precision_score(y_train_yes, y_pred_forest)

In [ ]:
recall_score(y_train_yes, y_pred_forest)

In [ ]:
sgd_clf.fit(X_train, y_train_yes) # y_train ET NON y_train_5
sgd_clf.predict(X_test)

In [ ]:
some_digit_scores = sgd_clf.decision_function(X_test)
class_max_index = np.argmax(some_digit_scores)

In [ ]:
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(max_iter=5, random_state=42))
ovo_clf.fit(X_train, y_train_yes)
ovo_clf.predict(X_test)

In [ ]:
forest_clf.fit(X_train, y_train_yes)
forest_clf.predict(X_test)

In [ ]:
forest_clf.predict_proba(X_test)

In [ ]:
cross_val_score(sgd_clf, X_train, y_train_yes, cv=3, scoring="accuracy")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, y_train_yes, cv=3, scoring="accuracy")

In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train_yes, cv=3)
conf_mx = confusion_matrix(y_train_yes, y_train_pred)
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

def plot_confusion_matrix(matrix):
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)

plot_confusion_matrix(conf_mx)

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx / row_sums

np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()

plot_confusion_matrix(norm_conf_mx)